In [1]:
from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

In [2]:
a =load_dataset("guangyil/yelp_short_v2")

In [3]:
tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")


In [4]:
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)

In [5]:
gpt2_pad_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.pad_token])[0]
gpt2_eos_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.eos_token])[0]
gpt2_bos_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.bos_token])[0]


In [6]:
gpt2_bos_token, gpt2_eos_token, gpt2_pad_token

(50258, 50259, 50257)

In [7]:
with open('/home/guangyi/LatentOps/data/datasets/yelp_data/train.shuf.merge', encoding='utf-8') as f:
    train = f.readlines()[:20000]
dataset = Dataset.from_dict({"text":train})

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 20000
})

In [9]:
def group_texts(examples):
    tokenized_0 = tokenizer_encoder(
            examples["text"].replace(" ''",'').replace("`` ",''), return_special_tokens_mask=True, truncation=True, max_length=512
            )
    tokenized_1 = tokenizer_decoder(
            ' '+examples["text"].replace(" ''",'').replace("`` ",''), return_special_tokens_mask=True, truncation=True, max_length=tokenizer_decoder.model_max_length
            )
    return {'bert_token':tokenized_0['input_ids'],'gpt2_token':[tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.bos_token])[0]]+tokenized_1['input_ids']+[tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.eos_token])[0]]}
# bookcorpus = Dataset.from_dict(bookcorpus[:100])
# preprocess dataset

In [10]:
dataset[0]['text'].replace(" ''",'').replace("`` ",'')

'excellent staff , comfortable , quiet rooms at a reasonable price .\n'

In [11]:
dataset[0]['text']

'excellent staff , comfortable , quiet rooms at a reasonable price .\n'

In [12]:
tokenizer_encoder(
            dataset[0]["text"].replace(" ''",'').replace("`` ",''), return_special_tokens_mask=True, truncation=True, max_length=512
            )

{'input_ids': [101, 6581, 3095, 1010, 6625, 1010, 4251, 4734, 2012, 1037, 9608, 3976, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

In [13]:
tokenizer_decoder(
            ' '+dataset[0]["text"].replace(" ''",'').replace("`` ",''), return_special_tokens_mask=True, truncation=True, max_length=tokenizer_decoder.model_max_length
            )

{'input_ids': [6275, 3085, 837, 6792, 837, 5897, 9519, 379, 257, 6397, 2756, 764, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [14]:
tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.bos_token])[0]

50258

In [15]:
tokenizer_decoder.bos_token

'<BOS>'

In [16]:
tokenizer_encoder.bos_token

In [17]:
train_eval_dataset =dataset.map(group_texts, batched=False, remove_columns=['text'],num_proc=10)

Map (num_proc=10):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [18]:
bert_pad_token = tokenizer_encoder.pad_token_id
bert_pad_token

0

In [19]:
def collate(examples):
    # Convert to Tensors and build dataset

    input_ids_bert = pad_sequence([torch.tensor(f['bert_token'], dtype=torch.long) for f in examples],
                                  batch_first=True, padding_value=bert_pad_token)
    input_ids_gpt = pad_sequence([torch.tensor(f['gpt2_token'], dtype=torch.long) for f in examples],
                                 batch_first=True, padding_value=gpt2_pad_token)
    try:
        token_lengths = torch.tensor([[len(f['bert_token']), len(f['gpt2_token'])] for f in examples],
                                     dtype=torch.long)
    except:
        token_lengths = torch.zeros((len(examples), 1091))
        for i in range(len(examples)):
            token_lengths[i, len(examples[i]['gpt2_token'])] = 1
    return (input_ids_bert, input_ids_gpt, token_lengths)

In [20]:
train_eval_dataloader = DataLoader(train_eval_dataset, num_workers=0,  pin_memory=True, collate_fn=collate,batch_size=64)


In [ ]:
def calc_rec_lgy(model_vae, encoder_tokenizer, decoder_tokenizer, args, eval_dataloader,ns=1):
    from modules import sample_sequence_conditional
    # eval_dataloader = build_dataload_and_cache_examples(args, [encoder_tokenizer, decoder_tokenizer], evaluate=True)
    count = 0
    result = defaultdict(str)
    ref = []
    cand = []
    for batch in tqdm(eval_dataloader, desc="Evaluating recontruction", disable=args.disable_bar):
        x0, x1, x_lengths = batch
        max_len_values, _ = x_lengths.max(0)
        x0 = x0[:, :max_len_values[0]]
        x1 = x1[:, :max_len_values[1]]
        x0 = x0.to(args.device)
        x1 = x1.to(args.device)
        x_lengths = x_lengths.to(args.device)
        context_tokens = decoder_tokenizer.encode(decoder_tokenizer.bos_token)
        with torch.no_grad():
            # text_x0 = encoder_tokenizer.decode(x0[0,:x_lengths[0,0]].tolist(), clean_up_tokenization_spaces=True)[0]
            # result["INPUT TEXT " + str(count)].append(text_x0)
            attention_mask = (x0 != encoder_tokenizer.pad_token_id).float()

            pooled_hidden_fea = model_vae.encoder(x0, attention_mask)[1]

            # Connect hidden feature to the latent space
            # latent_z, loss_kl = model_vae.connect(pooled_hidden_fea)
            mean, logvar = model_vae.encoder.linear(pooled_hidden_fea).chunk(2, -1)
            # latent_z = model_vae.reparameterize(mean, logvar, nsamples=1).squeeze(1)
            latent_z = mean.squeeze(1)

            past = latent_z
            out = sample_sequence_conditional(
                model=model_vae.decoder,
                context=context_tokens,
                past=past,
                length=x_lengths[0, 1],  # Chunyuan: Fix length; or use <EOS> to complete a sentence
                num_samples=latent_z.size(0),
                device=args.device,
                decoder_tokenizer=decoder_tokenizer,
                eos_id=model_vae.eos_token_id
            )

        for i in range(latent_z.size(0)):
            text_x0_ = decoder_tokenizer.decode(x1[i, :].tolist(), clean_up_tokenization_spaces=False).split(decoder_tokenizer.eos_token)[
                0].replace(decoder_tokenizer.bos_token, '').strip()
            text_x0_ = text_x0_.split()
            text_x1 = decoder_tokenizer.decode(out[i, :].tolist(), clean_up_tokenization_spaces=False).split(decoder_tokenizer.eos_token)[
                0].replace(decoder_tokenizer.bos_token, '').strip()
            text_x1 = text_x1.split()

            count += 1
            ref.append([text_x0_])
            cand.append(text_x1)

        if count > 1000:
            break
    bleu = corpus_bleu(ref, cand) * 100
    logger.info("  BLEU = %s", str(round(bleu, 2)))
    output_eval_file = os.path.join(args.output_dir, "eval_results_bleu.txt")
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    with open(output_eval_file, "w") as writer:
        writer.write("%s = %s\n" % ('bleu', str(bleu)))
    return {'bleu': bleu}